---
#  Mini ETL com Python para alimentar dashboard

Este notebook explica o processo de ETL (Extração, Transformação e Carga) dos dados de preços dos combustíveis no Brasil, utilizados em um dashboard desenvolvido no Power BI. O objetivo é consolidar diversos arquivos CSV semestrais em uma única base limpa e padronizada para análise visual, permitindo que novos dados sejam facilmente inseridos no fluxo sem necessidade de alterações manuais.

---
### Importação de bibliotecas

Utilizamos as seguintes bibliotecas:

- `os`: para manipulação e navegação entre arquivos e pastas do sistema.
- `pandas`: para leitura, transformação e consolidação dos dados tabulares (CSV).
- `chardet`: para detectar automaticamente o encoding de cada arquivo CSV, garantindo a leitura correta das informações.

In [1]:
import os
import pandas as pd
import chardet

---
### Listagem dos Arquivos CSV
Nesta etapa, criamos uma lista com os nomes de todos os arquivos localizados na pasta `DADOS`. Cada arquivo representa um período semestral de coleta de preços de combustíveis disponibilizado pela ANP.

In [2]:
caminho = r"C:\\Users\\hiago\\OneDrive\\Documentos\\CURSOS\\PORTIFOLIO\\DASHBOARD_PRECO_COMBUSTIVEL"
lista_arquivos = os.listdir(f"{caminho}/DADOS")
print(f"Arquivos encontrados: {len(lista_arquivos)}")

Arquivos encontrados: 36


---
### Consolidação dos Arquivos CSV

Nesta etapa, percorremos todos os arquivos da pasta `DADOS` e executamos as seguintes ações para consolidar os dados em um único DataFrame final:

- **Detectar o encoding de cada arquivo CSV**  
  Utilizamos a biblioteca `chardet` para identificar automaticamente a codificação do texto, evitando problemas com acentos e caracteres especiais.

- **Ler os arquivos CSV com o encoding correto**  
  Cada arquivo é lido individualmente com `pandas.read_csv`, utilizando o separador `;`.

- **Remover colunas desnecessárias**  
  Após a leitura, mantemos apenas as colunas definidas na lista `colunas_necess`, eliminando qualquer informação extra que não será utilizada na análise.

- **Concatenar os dados em um único DataFrame**  
  Cada arquivo processado é anexado ao `df_final`, formando uma base única consolidada para ser utilizada no Power BI.

Esse processo garante que, independentemente do número de arquivos existentes na pasta, os dados serão unificados de forma padronizada e consistente.

In [3]:
colunas_necess = [
    "Regiao - Sigla",
    "Estado - Sigla",
    "Municipio",
    "Revenda",
    "Produto",
    "Data da Coleta",
    "Valor de Venda",
    "Unidade de Medida",
    "Bandeira",
]

df_final = pd.DataFrame()

for arquivo in lista_arquivos:
    caminho_arquivo = f"{caminho}/DADOS/{arquivo}"
    
    # Detecta o encoding do arquivo
    with open(caminho_arquivo, 'rb') as f:
        resultado = chardet.detect(f.read(10000))
    
    # Leitura do CSV
    df = pd.read_csv(caminho_arquivo, sep=';', encoding=resultado['encoding'])
    
    # Remoção de colunas desnecessárias
    for coluna in df.columns:
        if coluna not in colunas_necess:
            df.drop(coluna, axis=1, inplace=True)
    
    # Concatenação no dataframe final
    df_final = pd.concat([df_final, df], ignore_index=True)

C:\Users\hiago\AppData\Local\Temp\ipykernel_18848\440578927.py:23: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho_arquivo, sep=';', encoding=resultado['encoding'])


---
### Conversão de Tipos de Dados
Nesta etapa, realizamos a conversão de tipo de dados nas colunas do DataFrame com o objetivo de **otimizar o uso de memória** e **melhorar o desempenho durante a análise no Power BI**.

As ações executadas são:

- **Iteração sobre todas as colunas do DataFrame**
  Verificamos cada coluna individualmente para aplicar o tipo de dado adequado.

- **Conversão para o tipo `category`**
  Todas as colunas, exceto `"Valor de Venda"`, são convertidas para o tipo `category`.  
  Esse tipo é ideal para colunas que possuem um número limitado de valores distintos (como nomes de estados, cidades, bandeiras, etc.).

- **Motivo da exceção**
  A coluna `"Valor de Venda"` é mantida com tipo numérico (float), pois será utilizada em cálculos e medidas no Power BI (como médias e variações).

Essa padronização reduz o uso de memória RAM e melhora o tempo de carregamento e resposta do Power BI ao manipular os dados.

In [4]:
for coluna in df_final.columns:
    if coluna != "Valor de Venda":
        df_final[coluna] = df_final[coluna].astype("category")

---
### Resultado Final

Ao final do processo, temos como resultado o `df_final`: um DataFrame consolidado, limpo e padronizado, contendo apenas os dados essenciais para a análise dos preços de combustíveis no Brasil.

Esse resultado cumpre exatamente o objetivo do projeto: **automatizar a preparação dos dados para visualização no Power BI**.  
Com essa base tratada, podemos integrar diretamente o Python ao Power BI, permitindo que o dashboard seja alimentado de forma automática e escalável sempre que novos arquivos forem adicionados à pasta de dados.

Assim, garantimos uma solução prática e reutilizável, que reduz o retrabalho manual e facilita a análise contínua da variação dos preços ao longo do tempo.

---

🔙 [**Voltar**](https://github.com/Hiagofb/DASHBOARD_COMBUSTIVEL) 